In [1]:
import matplotlib
%matplotlib notebook
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf
import numpy as np
import input_data

WORK_DIRECTORY = '/mnt/hdd1/thkim/dataset/MNIST'
NUM_CHANNELS = 1
PIXEL_DEPTH = 255
NUM_LABELS = 10
VALIDATION_SIZE = 5000  # Size of the validation set.
SEED = 66478  # Set to None for random seed.
BATCH_SIZE = 64
NUM_EPOCHS = 10
EVAL_BATCH_SIZE = 64
EVAL_FREQUENCY = 100  # Number of steps between evaluations.
mnist_width = 28
n_visible = mnist_width * mnist_width
n_hidden = 10
nBatch = 128
alpha_ = 0.5

conv1_weights = tf.Variable(
    tf.truncated_normal([5, 5, NUM_CHANNELS, 32],  # 5x5 filter, depth 32.
    stddev=0.1,
    seed=SEED))
conv1_biases = tf.Variable(tf.zeros([32]))
conv2_weights = tf.Variable(
    tf.truncated_normal([5, 5, 32, 64],
    stddev=0.1,
    seed=SEED))
conv2_biases = tf.Variable(tf.constant(0.1, shape=[64]))
fc1_weights = tf.Variable(  # fully connected, depth 512.
    tf.truncated_normal(
    [mnist_width // 4 * mnist_width // 4 * 64, 512],
    stddev=0.1,
    seed=SEED))
fc1_biases = tf.Variable(tf.constant(0.1, shape=[512]))
fc2_weights = tf.Variable(
    tf.truncated_normal([512, NUM_LABELS],
    stddev=0.1,
    seed=SEED))
fc2_biases = tf.Variable(tf.constant(0.1, shape=[NUM_LABELS]))

''' Data '''
mnist = input_data.read_data_sets("/mnt/hdd1/thkim/dataset/MNIST", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
teX = np.reshape(teX,(len(teX),mnist_width,mnist_width,NUM_CHANNELS))
trX = np.reshape(trX,(len(trX),mnist_width,mnist_width,NUM_CHANNELS))
teY = np.where(teY==1)[1]
trY = np.where(trY==1)[1]

''' create node for input data '''
#X = tf.placeholder("float32", [nBatch, n_visible], name='X')
X = tf.placeholder(
      "float32",
      shape=(nBatch, mnist_width, mnist_width, NUM_CHANNELS),name='X')
Y = tf.placeholder("int64", shape=(nBatch,),name='Y')

''' create nodes for hidden variables '''
def c_init(type='samples'):
    if type=='samples':
        C_init = tf.convert_to_tensor(trX[np.random.randint(0,len(trX),n_hidden)].T)
    elif type=='random':
        C_max = 4 * np.sqrt(6. / (n_hidden + n_hidden))
        C_init = tf.random_uniform(shape=[n_hidden, n_hidden],
                           minval=-C_max,maxval=C_max)
    else:
        C_init = None
    return C_init

C = tf.Variable(c_init('random'), name='C')

conv = tf.nn.conv2d(X,
                    conv1_weights,
                    strides=[1, 1, 1, 1],
                    padding='SAME')

relu = tf.nn.relu(tf.nn.bias_add(conv, conv1_biases))
pool = tf.nn.max_pool(relu,
                      ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1],
                      padding='SAME')
conv = tf.nn.conv2d(pool,
                    conv2_weights,
                    strides=[1, 1, 1, 1],
                    padding='SAME')
relu = tf.nn.relu(tf.nn.bias_add(conv, conv2_biases))
pool = tf.nn.max_pool(relu,
                      ksize=[1, 2, 2, 1],
                      strides=[1, 2, 2, 1],
                      padding='SAME')
pool_shape = pool.get_shape().as_list()
reshape = tf.reshape(
    pool,
    [pool_shape[0], pool_shape[1] * pool_shape[2] * pool_shape[3]])

hidden = tf.nn.relu(tf.matmul(reshape, fc1_weights) + fc1_biases,name='hidden')

FC = tf.matmul(hidden, fc2_weights) + fc2_biases
#if train:
#    hidden = tf.nn.dropout(hidden, 0.5, seed=SEED)


C2 = tf.expand_dims(tf.transpose(C),0)
X2 = tf.expand_dims(FC,1)

dist = tf.reduce_sum(tf.square(tf.sub(X2,C2)),2)
loss1 = tf.reduce_mean(tf.reduce_min(dist,1))
choice = tf.argmin(dist,1)

batch = tf.Variable(0)
learning_rate = tf.train.exponential_decay(
      1e-5,                # Base learning rate.
      batch * nBatch,  # Current index into the dataset.
      trX.shape[0],          # Decay step.
      0.95,                # Decay rate.
      staircase=True)
  # Use simple momentum for the optimization.



    
loss2 = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(FC, Y))

regularizers = (tf.nn.l2_loss(fc1_weights) + tf.nn.l2_loss(fc1_biases) +
                  tf.nn.l2_loss(fc2_weights) + tf.nn.l2_loss(fc2_biases))

alpha = tf.placeholder('float')
cost = (0.5-alpha)*loss1 + (0.5+alpha)*loss2 + 5e-4 * regularizers


train_op = tf.train.MomentumOptimizer(learning_rate,0.9).minimize(cost,global_step=batch)
#train_op = tf.train.GradientDescentOptimizer(1e-3).minimize(cost)  # construct an optimizer
#train_op = tf.train.AdadeltaOptimizer(1e-3).minimize(cost)
#train_op = tf.train.AdamOptimizer().minimize(cost)

''' Launch the graph in a session '''
sess = tf.Session()
C_acc = np.zeros((100,10,10))
tf.initialize_all_variables().run(session=sess)

print('-1', sess.run(cost, feed_dict={X: teX[:nBatch], Y:  teY[:nBatch],alpha:alpha_}))
C_acc[0] = sess.run(C)

for i in range(1000):
    if i== 20:
        idx = np.random.randint(0,len(trX),128)
        C.assign(sess.run(FC,feed_dict={X:trX[idx]})[:10])
    if i >= 20 and alpha_ >= 0.05:
        alpha_= alpha_ - 0.05
    if alpha_ < 0:
        alpha_ = 0
    cnt = 0
    for start, end in zip(range(0, len(trX), nBatch), range(nBatch, len(trX), nBatch)):
        cnt += 1
        input_ = trX[start:end]
        gt_ = trY[start:end]
        _,trainCost,l1,l2 = sess.run([train_op,cost,loss1,loss2], feed_dict={X: input_, Y: gt_, alpha:alpha_})
        if cnt % 100 == 0: print "Iter", cnt, trainCost, l1, l2
    if i<100: C_acc[i] = sess.run(C)
   
    out =  sess.run([cost,loss1,loss2,dist,choice], feed_dict={X: teX[:nBatch], Y: teY[:nBatch], alpha:alpha_})
    print '===================================Test iter %d %5.3f %5.3f %5.3f %f'%(i, out[0], out[1], out[2], alpha_)
    #print out[2]


('Extracting', '/mnt/hdd1/thkim/dataset/MNIST/train-images-idx3-ubyte.gz')


/usr/lib/python2.7/gzip.py:268: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
input_data.py:42: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


('Extracting', '/mnt/hdd1/thkim/dataset/MNIST/train-labels-idx1-ubyte.gz')
('Extracting', '/mnt/hdd1/thkim/dataset/MNIST/t10k-images-idx3-ubyte.gz')
('Extracting', '/mnt/hdd1/thkim/dataset/MNIST/t10k-labels-idx1-ubyte.gz')
('-1', 9.3404226)
Iter 100 5.7818 42.3646 2.6671
Iter 200 5.31755 41.56 2.20288
Iter 300 5.14726 44.2507 2.03263
Iter 400 4.96252 45.7553 1.84792
===================================Test iter 0 5.078 42.902 1.963 0.500000
Iter 100 4.9746 42.2676 1.86004
Iter 200 4.78539 43.0351 1.67086
Iter 300 4.66904 44.7163 1.55454
Iter 400 4.59508 46.552 1.48061
===================================Test iter 1 4.678 44.158 1.564 0.500000
Iter 100 4.66112 43.3532 1.54669
Iter 200 4.49448 45.6154 1.38007
Iter 300 4.37479 47.8315 1.26041
Iter 400 4.35589 49.7237 1.24153
===================================Test iter 2 4.425 47.443 1.311 0.500000
Iter 100 4.45662 46.4063 1.3423
Iter 200 4.30212 49.4044 1.18781
Iter 300 4.1772 51.9722 1.06292
Iter 400 4.18921 53.6947 1.07495
==============

KeyboardInterrupt: 

In [9]:
#sess.run(C)
i=2
sess.run([cost,dist,choice], feed_dict={X: teX[i:i+128], Y: teY[:nBatch], alpha:alpha_})


[4.8112035,
 array([[ 18.52417564,   8.85062027,  29.7660408 , ...,  20.50020981,
          15.21490288,  30.84970284],
        [ 22.74432755,   8.09888458,  34.54758835, ...,  20.02509308,
          17.19027138,  32.89435196],
        [ 21.35910606,  10.4706583 ,  35.51891327, ...,  18.40789795,
          19.30864143,  24.53445816],
        ..., 
        [ 20.67406082,   7.23206043,  34.647686  , ...,  19.00139618,
          19.30984688,  32.07115173],
        [ 19.72752571,   8.84017944,  30.79948997, ...,  21.95375061,
          15.18465996,  29.26916313],
        [ 18.98001289,   6.6545229 ,  35.66272736, ...,  23.46886063,
          19.93996811,  33.1199379 ]], dtype=float32),
 array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
 

In [13]:
sess.run([FC,,feed_dict={X: teX[i:i+128], Y: teY[:nBatch], alpha:alpha_})[0]

array([ 0.        ,  0.        ,  0.        ,  0.84242296,  0.        ,
        0.        ,  0.44587955,  0.        ,  0.        ,  0.        ,
        0.        ,  0.873519  ,  0.        ,  0.        ,  0.        ,
        0.21491808,  0.0541196 ,  0.        ,  0.        ,  0.87756169,
        0.81012118,  0.        ,  0.41548687,  0.        ,  0.        ,
        0.93266147,  1.1026715 ,  0.218925  ,  0.88959116,  0.        ,
        0.32422784,  0.        ,  0.        ,  0.64849174,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.46091956,
        0.06784925,  0.        ,  0.        ,  0.        ,  0.        ,
        1.62431395,  0.        ,  1.72730875,  0.        ,  0.77251101,
        1.16562068,  0.49161229,  0.42182651,  0.        ,  0.        ,
        0.61478388,  1.15518892,  0.        ,  0.        ,  0.        ,
        0.60864657,  0.        ,  0.557289  ,  0.        ,  0.        ,
        0.03772029,  0.1366367 ,  0.        ,  0.        ,  0.  

In [ ]:
t=C_acc
t=t.reshape(100,28,28,10) 

t2 = np.zeros((2800,280))
for i in range(100):
    for j in range(10):
        t2[i*28:(i+1)*28,j*28:(j+1)*28] = t[i,:,:,j]
plt.figure()
plt.imshow(t2[:,:],cmap = plt.get_cmap('gray'))
plt.show()

In [ ]:
from sklearn.cluster import KMeans

Kcents = 10
km = KMeans(init='random',n_clusters=Kcents, n_init=5, n_jobs=8)
#km = KMeans(init='k-means++',n_clusters=Kcents, n_init=5, n_jobs=8)
km.fit(trX)

In [ ]:
km.cluster_centers_.shape
plotcent(km.cluster_centers_.reshape((10,28,28)))

In [ ]:
import numpy as np
np.random.randint(0,len(trX),10)